In [1]:
import numpy as np
import sys
import os.path
import glob
import random
from collections import OrderedDict
from scipy.stats import lognorm, norm
from matplotlib import pyplot as plt
import mplhep as hep 
import pyhf
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
plt.style.use('loca_v_1_5/plot_style.txt')
from loca_v_1_5.loca import *
import time
plt.style.use(hep.style.CMS)

In [5]:
def e_e_CMS_search(lhco_file, m_min=0.0):

    process='p p --> e+ e-'

    search_description='''
    Recast of CMS search CMS PAS EXO-19-019:
    Search for a narrow resonance in high-mass dilepton final
    states in proton-proton collisions using 140 fb^−1
    of data at sqrt(s)=13 TeV'''
    
    #...Pre-cuts:
    basic_cuts = {                    
            'e'     : [35.0, 1e+10 ,  2.5],  # [pt_min, pt_max, abs(eta_max)] 
            'mu'    : [53.0, 1e+10 ,   2.4],
            'tau'   : [0.0 , 1e+10 ,  1e+10],
            'jet'   : [0.0 , 1e+10 ,  1e+10],
            'bjet'  : [0.0 , 1e+10 ,  1e+10],
            'met'   : [0.0 , 1e+10 ,  1e+10],
            'photon': [0.0 , 1e+10 ,  1e+10] 
             }

    signals=[]; effs=[]; xsecs=[]

    #...Signal region defintion:
    
  
    Category_electron = category(
                'nelectrons >= 2',
                'OS ee',
                'Mee > '+str(m_min))
        
    #...Observables:

    bins=[150.0, 160.0, 170.0, 
          180.0, 190.0, 200.0, 220.0, 240.0, 260.0, 280.0, 300.0, 320.0, 340.0, 
          360.0, 380.0, 400.0, 420.0, 440.0, 460.0, 480.0, 500.0, 520.0, 540.0, 
          560.0, 580.0, 600.0, 630.0, 660.0, 690.0, 720.0, 750.0, 780.0, 810.0, 
          840.0, 870.0, 900.0, 950.0, 1000.0, 1050.0, 1100.0, 1150.0, 1200.0, 
          1250.0, 1310.0, 1370.0, 1430.0, 1490.0, 1550.0, 1610.0, 1680.0, 1750.0, 
          1820.0, 1890.0, 1970.0, 2050.0, 2130.0, 2210.0, 2290.0, 2370.0, 2450.0, 
          2530.0, 2610.0, 2690.0, 2770.0, 2850.0, 2930.0, 3010.0, 3090.0, 3170.0, 
          3250.0, 3330.0, 3410.0, 3490.0, 3570.0, 3650.0, 3730.0, 3810.0, 3890.0, 
          3970.0, 4070.0, 4170.0, 4270.0, 4370.0, 4470.0, 4570.0, 4670.0, 4770.0, 
          4870.0, 4970.0, 5070.0, 5170.0, 5270.0, 5370.0, 5470.0, 5570.0, 5670.0, 
          5770.0, 5870.0, 5970.0 , 6070.0]
    
    Mee=spectrum(name='Mee')
    
    #...Apply cuts:
    
    lhco=open_lhco(file=lhco_file, cuts=basic_cuts, lhco_header=True)

    with lhco as events:   
        for N, event in events:

            #....start cuts

            Category_electron.start_cutflow(name='dielectron')

            if event.nleps>=2:                    
                e1=event.lepton_leading
                e2=event.lepton_subleading

                if (e1.is_electron and e2.is_electron):
                    Category_electron.apply_cut('nelectrons >= 2')

                    if e1.charge==-e2.charge:
                        Category_electron.apply_cut('OS ee')

                        m=e1.invM(e2) 

                        if m > m_min:
                            Category_electron.apply_cut('Mee > '+str(m_min))
                            Mee.extract_spectrum(m) 
                            lhco.parameters['signal_category']=Category_electron

        lhco.parameters[Mee.name]=Mee
        lhco.parameters['Mee_bins']=bins
        lhco.parameters['effs']=Mee.bin_data(bins)/float(lhco.parameters['nevents'])

    return lhco.parameters


In [6]:

def extract_results(data_files):

    FF={'FS':'scalar','FT':'tensor','FST':'scalar-tensor',
        'FVLL':'vector LL','FVLR':'vector LR','FD':'dipole','FV':'SM'}      
    
    kernel={}
    
    for lhco in glob.glob(data_files, recursive=True):        
        print('analyzing: '+lhco)
        res=e_e_CMS_search(lhco, m_min=150)
        info=res['run_tag'].split('_') 
        kernel[res['run_tag']]={}        
        kernel[res['run_tag']]['Form_Factor']=FF[info[0]]
        if info[1]=='SM':
            kernel[res['run_tag']]['FF_coeff']=None
        else:
            kernel[res['run_tag']]['FF_coeff']=list(info[1])
        kernel[res['run_tag']]['qq~']=info[2]+'~'
        kernel[res['run_tag']]['ll~']=info[3]
        kernel[res['run_tag']]['type']=info[4]+'*'+info[5]
        kernel[res['run_tag']]['mll_bin']=[float(info[-2]),float(info[-1])]
        kernel[res['run_tag']]['results']=res
            
    return kernel

def kernel_matrix(kernel,bins,output):  
    
    def get_bins(bins):
        d={}
        for i,b in enumerate(bins[:-1]):
            I=str([float(b),float(bins[i+1])])
            d[I]=i
        return d

    runs=[] 
    info={}
    for run in kernel.keys():
        x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
        if x not in runs:
            runs.append(x)
            info[x]=kernel[run]
    dic={}
    for key in runs:
        dic[key]={}
    
    bin_number=get_bins(bins)
    
    for r in runs:
        for run in kernel.keys():
            x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
            b=str(kernel[run]['mll_bin'])
            eff=kernel[run]['results']['effs']
            if (kernel[run]['type']=='A*Reg' or
               kernel[run]['type']=='Z*Reg' or
               kernel[run]['type']=='W*Reg'):
                xsec=round(kernel[run]['results']['xsec'],5)
            else:
                xsec=round(2*kernel[run]['results']['xsec'],5)
            pre=[str(xsec),
                 str(kernel[run]['mll_bin'][0]),
                 str(kernel[run]['mll_bin'][1])]
            
            eff_str=''
            
            for y in eff: eff_str+=str(y)+'\t'
            dic[x][bin_number[b]]='  '.join(pre+[eff_str]) 
                       
    for d in dic.keys():
        file=open(output+'/'+d+'.dat','w')
        file.write('#===============\n')
        file.write('# ll~ : '+ info[d]['ll~'] +'\n')
        file.write('# qq~ : '+ info[d]['qq~'] +'\n')
        ff=info[d]['Form_Factor'].split()
        file.write('# FF  : '+ ff[0] +'\n')
        
        # XY:
        if len(ff)==1:    
            file.write('# XY  : LL LR RL RR\n')
        else:
            if info[d]['Form_Factor'].split()[1]=='LL':
                file.write('# XY  : LL RR  \n')
            elif info[d]['Form_Factor'].split()[1]=='LR':
                file.write('# XY  : LR RL  \n')  
                
        if info[d]['FF_coeff']:
            coeff=info[d]['FF_coeff'][0].replace("'", "")+' '+info[d]['FF_coeff'][1].replace("'", "")
        else:
            coeff='None'
        file.write('# coef : '+coeff+'\n')
        file.write('# type : '+info[d]['type']+'\n')
        file.write('# Nevs : '+ str(info[d]['results']['nevents'])+'\n')
        file.write('#===============\n'  )
        file.write('#\n'  )
        file.write('# XSEC\tBIN_MIN\tBIN_MAX \t Kij \n')
        file.write('#\n'  )
        
        for i in list(bin_number.values()):
            try:
                file.write(dic[d][i]+'\n')
            except KeyError:
                inv_bin_number = {v: k for k, v in bin_number.items()}
                print('Error -->',d,':',inv_bin_number[i])
                file.write((str(None)+'\t')*len(bin_number)+'\n')
        file.close()
 

In [7]:
t0 = time.time()

FS_kernel  = extract_results(data_files='lhco_files/FS_lhco/*.gz')
FST_kernel = extract_results(data_files='lhco_files/FST_lhco/*.gz')
FT_kernel  = extract_results(data_files='lhco_files/FT_lhco/*.gz')
FVLL_kernel  = extract_results(data_files='lhco_files/FVLL_lhco/*.gz')
FVLR_kernel  = extract_results(data_files='lhco_files/FVLR_lhco/*.gz')
FD_kernel  = extract_results(data_files='lhco_files/FD_lhco/*.gz')
FV_kernel  = extract_results(data_files='lhco_files/FV_lhco/*.gz')

t1 = time.time()

print('time =',(t1-t0)/3600.)

analyzing: lhco_files/FS_lhco/FS_00_uc_ee_Reg_Reg_2750_3000_delphes_events.lhco.gz


/Users/dario/Dropbox/GitHub/High_pT_tails/e/e_e/SMEFT/loca_v_1_5/loca.py:212: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(self.invM2(obj))


analyzing: lhco_files/FS_lhco/FS_00_cu_ee_Reg_Reg_1200_1400_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_ds_ee_Reg_Reg_1800_2000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_uc_ee_Reg_Reg_2250_2500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cu_ee_Reg_Reg_3000_3500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bb_ee_Reg_Reg_800_900_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bb_ee_Reg_Reg_4000_15000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_db_ee_Reg_Reg_2250_2500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sd_ee_Reg_Reg_1800_2000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_db_ee_Reg_Reg_2750_3000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sd_ee_Reg_Reg_4000_15000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cc_ee_Reg_Reg_1400_1600_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_uc_ee_Reg_Reg_900_1000_delphes_events.lhco.gz
analy

analyzing: lhco_files/FS_lhco/FS_00_cc_ee_Reg_Reg_800_900_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sd_ee_Reg_Reg_1000_1200_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_ds_ee_Reg_Reg_4000_15000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cu_ee_Reg_Reg_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sd_ee_Reg_Reg_2250_2500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_db_ee_Reg_Reg_1800_2000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sb_ee_Reg_Reg_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sd_ee_Reg_Reg_2750_3000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cu_ee_Reg_Reg_2500_2750_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_uu_ee_Reg_Reg_2000_2250_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cu_ee_Reg_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_uu_ee_Reg_Reg_700_800_delphes_events.lhco.gz
analyzing: lh

analyzing: lhco_files/FS_lhco/FS_00_cc_ee_Reg_Reg_1000_1200_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bb_ee_Reg_Reg_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_db_ee_Reg_Reg_2000_2250_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cu_ee_Reg_Reg_3500_4000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_dd_ee_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cc_ee_Reg_Reg_4000_15000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_ds_ee_Reg_Reg_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_bb_ee_Reg_Reg_1800_2000_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_cc_ee_Reg_Reg_2500_2750_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sb_ee_Reg_Reg_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_uc_ee_Reg_Reg_2000_2250_delphes_events.lhco.gz
analyzing: lhco_files/FS_lhco/FS_00_sd_ee_Reg_Reg_800_900_delphes_events.lhco.gz
analyzing: 

...

analyzing: lhco_files/FD_lhco/FD_00_ds_ee_A_A_4000_15000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cu_ee_A_Z_2500_2750_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_db_ee_A_A_2250_2500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_dd_ee_A_Z_1800_2000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bb_ee_A_A_2000_2250_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uc_ee_A_Z_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cu_ee_Z_Z_2250_2500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_db_ee_A_A_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_ds_ee_Z_Z_1000_1200_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_dd_ee_A_A_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bs_ee_Z_Z_1400_1600_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cu_ee_Z_Z_2750_3000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_db_ee_A_Z_400_500_de

analyzing: lhco_files/FD_lhco/FD_00_bs_ee_A_A_2000_2250_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_dd_ee_Z_Z_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_ss_ee_A_Z_3500_4000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bb_ee_Z_Z_1400_1600_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uc_ee_Z_Z_3000_3500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uc_ee_A_Z_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cc_ee_A_Z_4000_15000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bd_ee_A_Z_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uu_ee_A_A_1000_1200_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_db_ee_Z_Z_600_700_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_db_ee_A_Z_2750_3000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cc_ee_Z_Z_800_900_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_sd_ee_Z_Z_2000_2250_delp

analyzing: lhco_files/FD_lhco/FD_00_uc_ee_Z_Z_1000_1200_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bd_ee_A_Z_4000_15000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_sb_ee_A_Z_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_db_ee_Z_Z_1200_1400_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_sd_ee_A_Z_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bs_ee_A_Z_1600_1800_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cu_ee_A_A_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_sd_ee_A_A_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cc_ee_Z_Z_2500_2750_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_ds_ee_A_A_1800_2000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_db_ee_Z_Z_3000_3500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_ds_ee_A_Z_800_900_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uu_ee_A_A_200_300_delphes

analyzing: lhco_files/FD_lhco/FD_00_bb_ee_A_Z_500_600_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uu_ee_A_Z_1200_1400_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uc_ee_A_A_2750_3000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cu_ee_Z_Z_1800_2000_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bd_ee_A_Z_700_800_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_sd_ee_Z_Z_1600_1800_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_cc_ee_A_A_300_400_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_bd_ee_A_A_400_500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uc_ee_A_A_2250_2500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_sb_ee_Z_Z_2000_2250_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_uu_ee_A_Z_3000_3500_delphes_events.lhco.gz
analyzing: lhco_files/FD_lhco/FD_00_dd_ee_A_Z_2750_3000_delphes_events.lhco.gz
analyzing: lhco_files/FV_lhco/FV_SM_ss_ee_Z_Z_1600_1800_delp

EOFError: Compressed file ended before the end-of-stream marker was reached

In [9]:
output=make_dir('Kernel_effs', overwrite=True)
bins=[200,300,400,500,600,700,800,900,1000,1200,1400,1600,1800,2000,2250,2500,2750,3000,3500,4000,15000]
kernel_matrix(FS_kernel,bins,output)
kernel_matrix(FT_kernel,bins,output)
kernel_matrix(FST_kernel,bins,output)
kernel_matrix(FVLL_kernel,bins,output)
kernel_matrix(FVLR_kernel,bins,output)
kernel_matrix(FD_kernel,bins,output)
kernel_matrix(FV_kernel,bins,output)


Error --> FVLL_00_db_ee_Reg_Reg : [1200.0, 1400.0]
Error --> FVLR_00_sb_ee_Reg_Reg : [1400.0, 1600.0]
Error --> FVLR_00_sb_ee_Reg_Reg : [1600.0, 1800.0]
Error --> FVLR_00_ds_ee_Reg_Reg : [1600.0, 1800.0]
Error --> FVLR_00_db_ee_Reg_Reg : [1400.0, 1600.0]
Error --> FVLR_00_db_ee_Reg_Reg : [1600.0, 1800.0]
Error --> FVLR_00_uc_ee_Reg_Reg : [1600.0, 1800.0]
Error --> FD_00_ss_ee_A_Z : [1600.0, 1800.0]
Error --> FD_00_bs_ee_Z_Z : [1800.0, 2000.0]
Error --> FD_00_bs_ee_A_Z : [1800.0, 2000.0]
Error --> FD_00_sb_ee_A_A : [1400.0, 1600.0]
Error --> FD_00_sb_ee_A_A : [1600.0, 1800.0]
Error --> FD_00_sb_ee_A_Z : [1600.0, 1800.0]
Error --> FD_00_ds_ee_Z_Z : [1600.0, 1800.0]
Error --> FD_00_cc_ee_A_A : [1800.0, 2000.0]
Error --> FD_00_sb_ee_Z_Z : [1400.0, 1600.0]
Error --> FD_00_sb_ee_Z_Z : [1600.0, 1800.0]
Error --> FD_00_ds_ee_A_A : [1600.0, 1800.0]
Error --> FD_00_bb_ee_A_Z : [1600.0, 1800.0]
Error --> FD_00_bb_ee_A_Z : [1800.0, 2000.0]
Error --> FD_00_db_ee_Z_Z : [1600.0, 1800.0]
Error --> FD_

NameError: name 'FV_kernel' is not defined